<a href="https://colab.research.google.com/github/chrisong88/cs238-final-project/blob/master/STV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install networkx

from copy import deepcopy
from random import randint
from itertools import combinations, permutations

import networkx as nx

from collections import OrderedDict
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [ ]:
# candidates, set of candidates
# ballots, list of ballots
# ballot: list of [quantity, [ranking]]
# original: candidate that you wish to clone
# Use Case: clones one candidate in a set of ballots, randomly chooses inter-clone ranking
def generate_clones(candidates, ballots, original):
    assert(original in candidates)
    new_candidates = candidates.copy()
    new_ballots = deepcopy(ballots)

    # generate clone and insert it into list of candidates
    clone = original + "x"
    while (clone in new_candidates):
        clone = clone + "x"
    new_candidates.add(clone)

    # insert clone into all ballots
    for ballot in new_ballots:
        index = ballot[1].index(original)
        ballot[1].insert(index + randint(0, 1), clone)
    return new_candidates, new_ballots

# non-exhaustive checking that candidates and ballots match up
def validate_inputs(candidates, ballots):
    assert(type(candidates) == set)
    assert(len(ballots) != 0)
    assert(len(ballots[0][1]) == len(candidates))
    assert(type(ballots) == list)
    assert(type(ballots[0]) == list)



In [ ]:
# IMPLEMENTING STV
# Input Types: set(string, ...), list[list[int, list[string, ...]]]
# Does not return
def run_election(candidates_original, ballots_original):
    candidates = deepcopy(candidates_original)
    ballots = deepcopy(ballots_original)
    validate_inputs(candidates, ballots)

    round = 0

    while (len(candidates) > 1):
        first_place = {}

        # initializing all first choice vote counts
        for candidate in candidates:
            first_place[candidate] = 0

        # counting up first choice vote counts
        for ballot in ballots:
            first_place[ballot[1][0]] += ballot[0]
        
        # finding the loser to eliminate
        loser = ""
        loser_count = -1
        for candidate in candidates:
            # break ties alphabetically (any deterministic process works)
            if loser_count == -1 or first_place[candidate] < loser_count or first_place[candidate] == loser_count and candidate < loser:
                loser = candidate
                loser_count = first_place[candidate]

        # print(f"Round {round} First Place Votes: {first_place}")
        
        assert(loser != "")
        assert(loser_count != -1)

        print(f"Candidate {loser} was eliminated")

        # Removing loser from ballots
        candidates.remove(loser)
        for ballot in ballots:
            ballot[1].remove(loser)

        round += 1
    
    for candidate in candidates:
        print (f"WINNER: Candidate {candidate}")
    
    


In [47]:
# Test Cases

candidates1 = {'A', 'B', 'C', 'D'}
ballots1 = [
    [5, ['A', 'B', 'C', 'D']],
    [4, ['B', 'C', 'A', 'D']],
    [3, ['C', 'D', 'A', 'B']],
    [1, ['C', 'A', 'B', 'D']]
]

candidates2, ballots2 = generate_clones(candidates1, ballots1, 'B')
candidates3, ballots3 = generate_clones(candidates2, ballots2, 'B')
candidates4, ballots4 = generate_clones(candidates3, ballots3, 'D')
candidates5, ballots5 = generate_clones(candidates4, ballots4, 'A')

In [51]:
# Test Cases from Schulz Wikipedia

candidates1 = {'A', 'B', 'C', 'D', 'E'}
ballots1 = [
    [5, ['A', 'C', 'B', 'E', 'D']],
    [5, ['A', 'D', 'E', 'C', 'B']],
    [8, ['B', 'E', 'D', 'A', 'C']],
    [3, ['C', 'A', 'B', 'E', 'D']],
    [7, ['C', 'A', 'E', 'B', 'D']],
    [2, ['C', 'B', 'A', 'D', 'E']],
    [7, ['D', 'C', 'E', 'B', 'A']],
    [8, ['E', 'B', 'A', 'D', 'C']]
]

candidates2, ballots2 = generate_clones(candidates1, ballots1, 'B')
candidates3, ballots3 = generate_clones(candidates2, ballots2, 'B')
candidates4, ballots4 = generate_clones(candidates3, ballots3, 'D')
candidates5, ballots5 = generate_clones(candidates4, ballots4, 'A')

In [ ]:

# STV Tests

test_cases = [(candidates1, ballots1), (candidates2, ballots2), (candidates3, ballots3), 
              (candidates4, ballots4), (candidates5, ballots5)]

for i in range(len(test_cases)):
    print("Running Test Case " + str(i))
    run_election(test_cases[i][0], test_cases[i][1])
    print()

Running Test Case 0
Candidate D was eliminated
Candidate B was eliminated
Candidate A was eliminated
Candidate E was eliminated
WINNER: Candidate C

Running Test Case 1
Candidate B was eliminated
Candidate D was eliminated
Candidate Bx was eliminated
Candidate A was eliminated
Candidate E was eliminated
WINNER: Candidate C

Running Test Case 2
Candidate B was eliminated
Candidate Bxx was eliminated
Candidate D was eliminated
Candidate Bx was eliminated
Candidate A was eliminated
Candidate E was eliminated
WINNER: Candidate C

Running Test Case 3
Candidate B was eliminated
Candidate Bxx was eliminated
Candidate Dx was eliminated
Candidate D was eliminated
Candidate Bx was eliminated
Candidate A was eliminated
Candidate E was eliminated
WINNER: Candidate C

Running Test Case 4
Candidate B was eliminated
Candidate Bxx was eliminated
Candidate Dx was eliminated
Candidate A was eliminated
Candidate D was eliminated
Candidate Bx was eliminated
Candidate Ax was eliminated
Candidate E was elim

In [49]:
# IMPLEMENTING RANKED PAIRS

def run_ranked_pairs(candidates_original, ballots_original, logging=""):
    candidates = deepcopy(candidates_original)
    ballots = deepcopy(ballots_original)
    validate_inputs(candidates, ballots)

    score = {}

    for combo in combinations(candidates, 2):
        score[tuple(sorted(combo))] = 0

    # print(score)

    for count, ranking in ballots:
        for i in range(len(ranking)):
            for j in range(i+1, len(ranking)):
                if (ranking[i] < ranking[j]):
                    score[(ranking[i], ranking[j])] += count
                    # if (ranking[i] == 'B' and ranking[j] == 'C'):
                    #     print(f"BC change: {count}")
                else:
                    score[(ranking[j], ranking[i])] -= count
                    # if (ranking[j] == 'B' and ranking[i] == 'C'):
                    #     print(f"BC change: {-count}")

    print(score)
    new_d = OrderedDict(sorted(score.items(), key=lambda t: t[0]))
    print(new_d)
    
    size = len(ranking)

    # Create DictVectorizer object
    dictvectorizer = DictVectorizer(sparse=False)

    # Convert dictionary into feature matrix
    features = dictvectorizer.fit_transform(new_d)
    print(features)

    pair_scores = []

    for pair in score:
        if score[pair] >= 0:
            pair_scores.append((score[pair], pair))
        else:
            pair_scores.append((-score[pair], (pair[1], pair[0])))

    pair_scores = sorted(pair_scores)

    pair_scores.reverse()

    # print(pair_scores)

    G = nx.DiGraph()

    i = 0
    while len(candidates) > 1:
        assert(i < len(pair_scores))
        pair = pair_scores[i][1]
        G.add_edge(pair[0], pair[1])
        if len(list(nx.simple_cycles(G))) > 0:
            G.remove_edge(pair[0], pair[1])
            if logging == "edges":
                print(f"Discarded edge {pair[0], pair[1]}")
        else:
            if logging == "edges":
                print(f"Added edge {pair[0], pair[1]}")
            if (pair[1] in candidates):
                candidates.remove(pair[1])
                if logging == "candidates":
                    print(f"Eliminated candidate {pair[1]}")
        i += 1

    for candidate in candidates:
        print(f"WINNER: {candidate}")

        m = nx.adjacency_matrix(G)
        print(m.todense())

In [52]:

# run_ranked_pairs(candidates1, ballots1, "edges")

test_cases = [(candidates1, ballots1), (candidates2, ballots2), (candidates3, ballots3), 
              (candidates4, ballots4), (candidates5, ballots5)]

for i in range(len(test_cases)):
    print("Running Test Case " + str(i))
    run_ranked_pairs(test_cases[i][0], test_cases[i][1], "edges")
    print()

Running Test Case 0
{('A', 'D'): 15, ('C', 'D'): -11, ('D', 'E'): -17, ('B', 'D'): 21, ('A', 'C'): 7, ('A', 'E'): -1, ('A', 'B'): -5, ('C', 'E'): 3, ('B', 'C'): -13, ('B', 'E'): -9}
OrderedDict([(('A', 'B'), -5), (('A', 'C'), 7), (('A', 'D'), 15), (('A', 'E'), -1), (('B', 'C'), -13), (('B', 'D'), 21), (('B', 'E'), -9), (('C', 'D'), -11), (('C', 'E'), 3), (('D', 'E'), -17)])
[[ -5.   7.  15.  -1. -13.  21.  -9. -11.   3. -17.]]
Added edge ('B', 'D')
Added edge ('E', 'D')
Added edge ('A', 'D')
Added edge ('C', 'B')
Discarded edge ('D', 'C')
Added edge ('E', 'B')
Added edge ('A', 'C')
Discarded edge ('B', 'A')
Added edge ('C', 'E')
WINNER: A
[[0 1 0 0 0]
 [0 0 0 0 0]
 [1 1 0 0 0]
 [0 1 0 0 1]
 [1 0 1 0 0]]

Running Test Case 1
{('A', 'D'): 15, ('C', 'D'): -11, ('Bx', 'D'): 21, ('D', 'E'): -17, ('B', 'D'): 21, ('A', 'C'): 7, ('A', 'Bx'): -5, ('A', 'E'): -1, ('A', 'B'): -5, ('Bx', 'C'): -13, ('C', 'E'): 3, ('B', 'C'): -13, ('Bx', 'E'): -9, ('B', 'Bx'): 23, ('B', 'E'): -9}
OrderedDict([(('A'

In [ ]:
ballots1

[[5, ['A', 'B', 'C', 'D', 'E']],
 [5, ['A', 'D', 'E', 'C', 'B']],
 [8, ['B', 'E', 'D', 'A', 'C']],
 [3, ['C', 'A', 'B', 'E', 'D']],
 [7, ['C', 'A', 'E', 'B', 'D']],
 [2, ['C', 'B', 'A', 'D', 'E']],
 [7, ['D', 'C', 'E', 'B', 'A']],
 [8, ['E', 'B', 'A', 'D', 'C']]]

In [ ]:
# IMPLEMENTING SCHULTZE

# generalizable minimum function
def get_min(first_tuple, second_tuple):
    if first_tuple[0] < second_tuple[0]:
        return first_tuple
    else:
        return second_tuple


def run_schultze(candidates_original, ballots_original):
    candidates = deepcopy(candidates_original)
    ballots = deepcopy(ballots_original)
    validate_inputs(candidates, ballots)

    score = {}

    for pair in permutations(candidates, 2):
        score[pair] = 0

    # print(score)
    
    for count, ranking in ballots:
        for i in range(len(ranking)):
            for j in range(i+1, len(ranking)):
                score[(ranking[i], ranking[j])] += count

    P = {}
    pred = {}

    candidates = list(candidates)

    # Initialization
    for i in range(len(candidates)):
        for j in range(len(candidates)):
            if i != j:
                P[candidates[i], candidates[j]] = (score[candidates[i], candidates[j]], score[candidates[j], candidates[i]])
                pred[candidates[i], candidates[j]] = candidates[i]

    # Calculation of Strength of Strongest Paths
    for i in range(len(candidates)):
        for j in range(len(candidates)):
            for k in range(len(candidates)):
                if i != j and j != k and i != k:
                    # winning votes -> 
                    if P[candidates[j], candidates[k]][0] < get_min(P[candidates[j], candidates[i]], P[candidates[i], candidates[k]])[0]:
                        P[candidates[j], candidates[k]] = get_min(P[candidates[j], candidates[i]], P[candidates[i], candidates[k]])
                        pred[candidates[j], candidates[k]] = pred[candidates[i], candidates[k]]

    winner = {}

    # Calculation of Winners
    for i in range(len(candidates)):
        winner[candidates[i]] = True
        for j in range(len(candidates)):
            if i != j and P[candidates[j], candidates[i]][0] > P[candidates[i], candidates[j]][0]:
                winner[candidates[i]] = False

    for candidate in winner:
        if winner[candidate]:
            print(f"WINNER: {candidate}")

In [ ]:
# run_schultze(candidates1, ballots1)

for i in range(len(test_cases)):
    print("Running Test Case " + str(i))
    run_schultze(test_cases[i][0], test_cases[i][1])
    print()

Running Test Case 0
WINNER: E

Running Test Case 1
WINNER: E

Running Test Case 2
WINNER: E

Running Test Case 3
WINNER: E

Running Test Case 4
WINNER: E



In [ ]:
for combo in combinations([1, 2, 3, 4], 2):
    print(combo)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [ ]:
# sorted(['A', 'B', 'C', 'D', 'E', 'F', 'Bx'])

['A', 'B', 'Bx', 'C', 'D', 'E', 'F']

In [ ]:
tuple(['A', 'B', 'C'])

('A', 'B', 'C')

In [ ]:
# sorted((1, 3))

[1, 3]

In [ ]:
'A' < 'B'

True